In [1]:
from rosetta import *
rosetta.init()

In [2]:
pose1 = pose_from_pdb('/home/steve/Desktop/projects/active_site_rms_pyrosetta/pose1.pdb')
pose2 = pose_from_pdb('/home/steve/Desktop/projects/active_site_rms_pyrosetta/pose2.pdb')

In [3]:
print pose1.residue(1).xyz('CA')
print pose2.residue(1).xyz('CA')

      58.50600000000000       42.69900000000000       28.08300000000000
      61.92500000000000       38.90400000000000       24.13600000000000


Trying to use the TMalign algorithm to do my superposition of fragments/proteins


In [4]:
tm = rosetta.protocols.hybridization.TMalign()
tm.apply(pose1, pose2)

0

In [22]:
longest = max(pose1.n_residue()+1, pose2.n_residue()+1)
print longest              # need to normalize by one of the lengths of the proteins
shortest = min(pose1.n_residue()+1, pose2.n_residue()+1)
print tm.TMscore(longest)  # yields tmscore of .957 ( or very similar in structure)

213
0.957125933343


In [14]:
# but the tmalign object doesn't actually move the pose....
print pose1.residue(1).xyz('CA')
print pose2.residue(1).xyz('CA')

      58.50600000000000       42.69900000000000       28.08300000000000
      61.92500000000000       38.90400000000000       24.13600000000000


From /src/protocols/hybridization/HybridizeProtocol.cc, lines 1439 to 1466
demonstrates how to run tmalign and then do the partial threading to actually move the pose

```c++
// get tmalign sequence mapping
		TMalign tm_align;
		std::string seq_pose, seq_ref, aligned;
		int reval = tm_align.apply(pose, ref_pose, residue_list, ref_residue_list);
		if ( reval != 0 ) continue;  // TO DO: remove this domain

		tm_align.alignment2AtomMap(pose, ref_pose, residue_list, ref_residue_list, n_mapped_residues, atom_map);
		tm_align.alignment2strings(seq_pose, seq_ref, aligned);

		using namespace ObjexxFCL::format;
		Size norm_length = residue_list.size() < ref_residue_list.size() ? residue_list.size():ref_residue_list.size();
		TR << "Align domain with TMscore of " << F(8,3,tm_align.TMscore(norm_length)) << std::endl;
		TR << seq_pose << std::endl;
		TR << aligned << std::endl;
		TR << seq_ref << std::endl;

		if ( n_mapped_residues < 6 ) continue;  // TO DO: remove this domain

		// add in ligand residues
		for ( core::Size i=last_protein_residue+1; i<=pose.total_residue(); ++i ) {
			core::Size res_controlling_i = ligres_map[i];
			for ( core::Size iloop=1; iloop<=domains[i_domain].num_loop(); ++iloop ) {
				if ( res_controlling_i < domains[i_domain][iloop].start() || res_controlling_i > domains[i_domain][iloop].stop() ) continue;
				residue_list.push_back(i);
			}
		}

		partial_align(pose, ref_pose, atom_map, residue_list, true, aln_cutoffs, min_coverage);
        ```

So I need to make run this method first:

tm.alignment2AtomMap(pose, ref_pose, residue_list, ref_residue_list, n_mapped_residues, atom_map);

which results below

1413    core::id::AtomID_Map< core::id::AtomID > atom_map;

1414    core::pose::initialize_atomid_map( atom_map, pose, core::id::BOGUS_ATOM_ID )

In [16]:
atommap = rosetta.core.id.AtomID_Map_T_core_id_AtomID_T?

In [31]:
#atommap = rosetta.core.id.AtomID_Map_T_core_id_AtomID_T( rosetta.core.id.AtomID() )
atommap = rosetta.core.id.AtomID_Map_AtomID()

In [20]:
#rosetta.core.pose.initialize_atomid_map_AtomID?
rosetta.core.pose.initialize_atomid_map( atommap, pose1 )

In [25]:
## create lists of ints for the residues to overlay

pose1list = [ x for x in xrange(1,shortest)]
pose2list = [ x for x in xrange(1,shortest)]
n_mapped_res = 0

In [32]:
tm.alignment2AtomMap(pose1,pose2, pose1list, pose2list, n_mapped_res, atommap )

ArgumentError: Python argument types in
    TMalign.alignment2AtomMap(TMalign, Pose, Pose, list, list, int, AtomID_Map_AtomID)
did not match C++ signature:
    alignment2AtomMap(protocols::hybridization::TMalign {lvalue}, core::pose::Pose pose, core::pose::Pose ref_pose, std::list<unsigned long, std::allocator<unsigned long> > residue_list, std::list<unsigned long, std::allocator<unsigned long> > ref_residue_list, unsigned long {lvalue} n_mapped_residues, core::id::AtomID_Map<core::id::AtomID> {lvalue} atom_map)
    alignment2AtomMap(protocols::hybridization::TMalign {lvalue}, core::pose::Pose pose1, core::pose::Pose pose2, unsigned long {lvalue} n_mapped_residues, core::id::AtomID_Map<core::id::AtomID> {lvalue} atom_map)